In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
# makes figures look better in Jupyter
sns.set_context("talk")

In [ ]:
import sys 
from pathlib import Path 
sys.path.insert(0, str(Path.cwd().parents[1]) + "/omniplate_platereader_analysis_code")

In [ ]:
import omniplate as om

In [ ]:
p= om.platereader(wdir= '../../../../data/raw_platereader/promoter_terminator_swaps/mTurq_collection/')

In [ ]:
p.load('20201204-Pro-ter_mTurq2-collection_n5.xlsx', '20201112-pro-ter_mTurq_contents.xls')

In [ ]:
p.r.head()

In [ ]:
p.r.columns

In [ ]:
p.s

In [ ]:
p.plot(y= 'OD', plate= True)
p.plot(y= 'mTurq-60', plate= True)

In [ ]:
p.correctOD()

In [ ]:
p.plot(y= 'mTurq-60', by= 'well')


In [ ]:
sns.relplot(x= 'time', y= 'OD', data= p.r, kind= 'line', hue= 'well', 
            col= 'strain')

In [ ]:
p.getstats()

In [ ]:
p.sc[['experiment','strain', 'max gr', 'max gr var']].head()

In [ ]:
p.plot(y= 'gr', hue= 'strain')

In [ ]:
p.correctauto(['mTurq-60'], refstrain= 'POT1-ccdB')

In [ ]:
p.exportdf()

In [ ]:
q= om.platereader(wdir= 'data/', ls= False)
q.importdf('20201204-Pro-ter_mTurq2-collection_n5')

In [ ]:
e= '20201204-Pro-ter_mTurq2-collection_n5'
# store results as an array of dictionaries to eventually convert into a dataframe
results= []
for s in p.allstrains[e]:
        # find the time of local maximum growth rate for the condition
        tm= p.sc.query('strain == @s')['time of local max gr'].values[0]
        # take the relevant sub-dataframe for the condition
        df= p.s.query('strain == @s')
        # find GPF mean at time tm
        i= np.argmin(np.abs(df['time'].values - tm))
        results.append({'c-mTurq-60 per OD at max local gr' : df['c-mTurq-60perod'][df.index[i]],
                     'strain' : s})
# convert to dataframe
rdf= pd.DataFrame(results)
# add to existing dataframe by experiment, condition, and strain
p.sc= pd.merge(p.sc, rdf, how= 'outer')

In [ ]:
p.sc[['strain','c-mTurq-60 per OD at max local gr']].to_csv("../../../../data/norm_platereader/promoter_terminator_swaps/mTurq_collection/pro-mTurq-ter_swaps_n=5.csv")